# 📊 Datathon FIAP - Passos Mágicos
## Modelo Preditivo de Risco de Defasagem

Este notebook desenvolve e compara múltiplos modelos de Machine Learning para identificar alunos em risco de defasagem educacional.

**Objetivo:** Criar um modelo preditivo que identifique padrões nos indicadores de desempenho e variáveis contextuais que permitam prever alunos em risco de defasagem escolar.

**Classificação de Risco:**
- **Sem Risco**: Aluno em fase adequada ou adiantado (D ≥ 0)
- **Com Risco**: Aluno atrasado em relação à fase ideal (D < 0)

**Features utilizadas:**
- Indicadores PEDE: IDA, IEG, IAA, IPS, IPV
- Notas: Matemática, Português, Inglês
- Contextuais: Idade, Ano de Ingresso, Gênero, Instituição de Ensino

**Autor:** Leandro Leme Crespo

---

## 1. Configuração do Ambiente

In [ ]:
# Baixar dados do GitHub (executar apenas no Google Colab)
import urllib.request
import os

os.makedirs('data', exist_ok=True)
os.makedirs('streamlit', exist_ok=True)

url = 'https://github.com/LeandroCrespo/datathon-passos-magicos/raw/main/data/BASE_DE_DADOS_PEDE_2024_DATATHON.xlsx'
filename = 'data/BASE_DE_DADOS_PEDE_2024_DATATHON.xlsx'

print('📥 Baixando dados do GitHub...')
urllib.request.urlretrieve(url, filename)
print('✅ Dados baixados com sucesso!')

In [ ]:
# Instalar bibliotecas necessárias
!pip install openpyxl scikit-learn imbalanced-learn xgboost -q
print('✅ Bibliotecas instaladas!')

In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score,
                             precision_score, recall_score, f1_score, roc_auc_score, roc_curve)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import pickle

plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

print('✅ Bibliotecas importadas com sucesso!')

## 2. Carregamento e Preparação dos Dados

In [ ]:
# Carregar dados
CAMINHO_ARQUIVO = 'data/BASE_DE_DADOS_PEDE_2024_DATATHON.xlsx'

xlsx = pd.ExcelFile(CAMINHO_ARQUIVO)
print(f'📋 Planilhas disponíveis: {xlsx.sheet_names}')

all_data = []
for sheet in xlsx.sheet_names:
    df_year = pd.read_excel(xlsx, sheet_name=sheet)
    df_year.columns = [c.upper() for c in df_year.columns]
    
    # Padronizar colunas
    if 'DEFAS' in df_year.columns and 'DEFASAGEM' not in df_year.columns:
        df_year = df_year.rename(columns={'DEFAS': 'DEFASAGEM'})
    if 'MATEM' in df_year.columns:
        df_year = df_year.rename(columns={'MATEM': 'MAT'})
    if 'PORTUG' in df_year.columns:
        df_year = df_year.rename(columns={'PORTUG': 'POR'})
    if 'INGLÊS' in df_year.columns:
        df_year = df_year.rename(columns={'INGLÊS': 'ING'})
    if 'GÊNERO' in df_year.columns:
        df_year['GÊNERO'] = df_year['GÊNERO'].replace({'Menina': 'Feminino', 'Menino': 'Masculino'})
    
    if 'DEFASAGEM' in df_year.columns:
        df_year['ANO_PEDE'] = sheet
        all_data.append(df_year)
        print(f'   {sheet}: {len(df_year)} registros')

df = pd.concat(all_data, ignore_index=True)
print(f'\n📊 Total de registros: {len(df):,}')

In [ ]:
# Definir features
# NOTA: Excluímos INDE e IAN para evitar data leakage
features_numericas = ['IDA', 'IEG', 'IAA', 'IPS', 'IPV', 'IDADE', 'ANO INGRESSO', 'MAT', 'POR', 'ING']
features_categoricas = ['GÊNERO', 'INSTITUIÇÃO DE ENSINO']

# Converter numéricas
for col in features_numericas:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Converter categóricas
le_dict = {}
for col in features_categoricas:
    if col in df.columns:
        df[col] = df[col].fillna('Desconhecido')
        le = LabelEncoder()
        df[col + '_ENC'] = le.fit_transform(df[col].astype(str))
        le_dict[col] = le
        print(f'{col}: {dict(zip(le.classes_, range(len(le.classes_))))}')

print('\n✅ Features preparadas')

## 3. Definição da Variável Alvo

**D = Fase Efetiva - Fase Ideal**

| Defasagem (D) | Classificação |
|---------------|---------------|
| D ≥ 0 | Sem Risco |
| D < 0 | Com Risco |

In [ ]:
# Criar variável alvo
df['DEFASAGEM'] = pd.to_numeric(df['DEFASAGEM'], errors='coerce')

def classificar_risco(d):
    if pd.isna(d): return None
    if d >= 0: return 0  # Sem Risco
    else: return 1  # Com Risco

df['CLASSE_RISCO'] = df['DEFASAGEM'].apply(classificar_risco)

# Selecionar features disponíveis
features_final = []
for col in features_numericas:
    if col in df.columns:
        features_final.append(col)
for col in features_categoricas:
    if col + '_ENC' in df.columns:
        features_final.append(col + '_ENC')

print(f'Features utilizadas ({len(features_final)}): {features_final}')

# Remover linhas com valores nulos
df_model = df.dropna(subset=['CLASSE_RISCO'] + features_final)
df_model['CLASSE_RISCO'] = df_model['CLASSE_RISCO'].astype(int)

print(f'\nRegistros válidos: {len(df_model):,}')
print(f'Sem Risco (0): {(df_model["CLASSE_RISCO"] == 0).sum()} ({(df_model["CLASSE_RISCO"] == 0).mean()*100:.1f}%)')
print(f'Com Risco (1): {(df_model["CLASSE_RISCO"] == 1).sum()} ({(df_model["CLASSE_RISCO"] == 1).mean()*100:.1f}%)')

In [ ]:
# Visualizar distribuição
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

cores = ['#2ecc71', '#e74c3c']
labels = ['Sem Risco', 'Com Risco']
valores = df_model['CLASSE_RISCO'].value_counts().sort_index()

bars = axes[0].bar(labels, valores.values, color=cores, edgecolor='black')
axes[0].set_title('Distribuição das Classes de Risco', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Quantidade de Alunos')
for bar, val in zip(bars, valores.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, f'{val}', ha='center', fontsize=12, fontweight='bold')

axes[1].pie(valores.values, labels=labels, colors=cores, autopct='%1.1f%%', startangle=90, explode=(0, 0.05))
axes[1].set_title('Proporção das Classes de Risco', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('distribuicao_risco.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Análise dos Padrões de Risco

In [ ]:
# Média dos indicadores por classe
indicadores = ['IDA', 'IEG', 'IAA', 'IPS', 'IPV']
media_por_classe = df_model.groupby('CLASSE_RISCO')[indicadores].mean()
media_por_classe.index = ['Sem Risco', 'Com Risco']

print('📊 Média dos Indicadores por Classe de Risco:')
print(media_por_classe.round(2))

print('\n📊 Diferença (Sem Risco - Com Risco):')
diff = media_por_classe.loc['Sem Risco'] - media_por_classe.loc['Com Risco']
for feat in diff.sort_values(ascending=False).index:
    print(f'   {feat}: +{diff[feat]:.2f}')

In [ ]:
# Análise de Idade
print('📊 Análise de Idade:')
print(f'   Sem Risco - Média: {df_model[df_model["CLASSE_RISCO"]==0]["IDADE"].mean():.1f} anos')
print(f'   Com Risco - Média: {df_model[df_model["CLASSE_RISCO"]==1]["IDADE"].mean():.1f} anos')
print('\n→ Alunos mais velhos têm MAIOR risco de defasagem')
print('  (A idade funciona como multiplicador - acúmulo de comportamentos ao longo do tempo)')

## 5. Preparação dos Dados para Modelagem

In [ ]:
# Separar features e target
X = df_model[features_final].values
y = df_model['CLASSE_RISCO'].values

print(f'Shape X: {X.shape}')
print(f'Shape y: {y.shape}')

# Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f'\nTreino: {len(X_train)} amostras')
print(f'Teste: {len(X_test)} amostras')

In [ ]:
# Normalizar
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('✅ Dados normalizados')

# SMOTE para balancear
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

print(f'\nApós SMOTE:')
print(f'   Treino original: {len(X_train_scaled)} amostras')
print(f'   Treino balanceado: {len(X_train_balanced)} amostras')

## 6. Comparação de Modelos de Machine Learning

Vamos comparar **10 algoritmos diferentes** para encontrar o melhor modelo:

| Categoria | Modelos |
|-----------|--------|
| **Lineares** | Logistic Regression |
| **Baseados em Distância** | K-Nearest Neighbors (KNN), SVM |
| **Baseados em Árvore** | Decision Tree, Random Forest, Gradient Boosting, XGBoost, AdaBoost |
| **Redes Neurais** | MLP (Multi-Layer Perceptron) |
| **Ensemble** | Voting Classifier |

In [ ]:
# Definir todos os modelos a serem testados
modelos = {
    # Modelos Lineares
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    
    # Baseados em Distância
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42),
    
    # Baseados em Árvore
    'Decision Tree': DecisionTreeClassifier(max_depth=10, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=200, max_depth=7, learning_rate=0.1, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=200, max_depth=7, learning_rate=0.1, random_state=42, use_label_encoder=False, eval_metric='logloss'),
    'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=42),
    
    # Redes Neurais
    'MLP (Rede Neural)': MLPClassifier(hidden_layer_sizes=(128, 64, 32), max_iter=500, random_state=42)
}

print(f'Total de modelos a testar: {len(modelos)}')
print('='*70)

In [ ]:
# Treinar e avaliar cada modelo
print('\n' + '='*70)
print('TREINAMENTO E AVALIAÇÃO DOS MODELOS')
print('='*70 + '\n')

resultados = []

for nome, modelo in modelos.items():
    print(f'Treinando {nome}...')
    
    # Treinar
    modelo.fit(X_train_balanced, y_train_balanced)
    
    # Predições
    y_pred = modelo.predict(X_test_scaled)
    y_proba = modelo.predict_proba(X_test_scaled)[:, 1]
    
    # Métricas
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    
    # Cross-validation
    cv_scores = cross_val_score(modelo, X_train_balanced, y_train_balanced, cv=5, scoring='accuracy')
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    
    resultados.append({
        'Modelo': nome,
        'Acurácia': acc,
        'Precisão': prec,
        'Recall': rec,
        'F1-Score': f1,
        'AUC-ROC': auc,
        'CV Mean': cv_mean,
        'CV Std': cv_std,
        'modelo_obj': modelo,
        'y_pred': y_pred,
        'y_proba': y_proba
    })
    
    print(f'   Acurácia: {acc*100:.2f}% | AUC-ROC: {auc*100:.2f}% | CV: {cv_mean*100:.2f}% (±{cv_std*100:.2f}%)')

print('\n✅ Todos os modelos treinados!')

In [ ]:
# Criar Ensemble Voting com os 3 melhores modelos
print('\nCriando Ensemble Voting...')

voting = VotingClassifier(
    estimators=[
        ('rf', modelos['Random Forest']),
        ('gb', modelos['Gradient Boosting']),
        ('mlp', modelos['MLP (Rede Neural)'])
    ],
    voting='soft'
)

voting.fit(X_train_balanced, y_train_balanced)
y_pred_ens = voting.predict(X_test_scaled)
y_proba_ens = voting.predict_proba(X_test_scaled)[:, 1]

acc_ens = accuracy_score(y_test, y_pred_ens)
prec_ens = precision_score(y_test, y_pred_ens)
rec_ens = recall_score(y_test, y_pred_ens)
f1_ens = f1_score(y_test, y_pred_ens)
auc_ens = roc_auc_score(y_test, y_proba_ens)
cv_ens = cross_val_score(voting, X_train_balanced, y_train_balanced, cv=5, scoring='accuracy')

resultados.append({
    'Modelo': 'Ensemble Voting',
    'Acurácia': acc_ens,
    'Precisão': prec_ens,
    'Recall': rec_ens,
    'F1-Score': f1_ens,
    'AUC-ROC': auc_ens,
    'CV Mean': cv_ens.mean(),
    'CV Std': cv_ens.std(),
    'modelo_obj': voting,
    'y_pred': y_pred_ens,
    'y_proba': y_proba_ens
})

print(f'   Acurácia: {acc_ens*100:.2f}% | AUC-ROC: {auc_ens*100:.2f}%')

## 7. Ranking dos Modelos

In [ ]:
# Criar DataFrame com resultados
df_resultados = pd.DataFrame(resultados)

# Ordenar por Acurácia
df_ranking = df_resultados[['Modelo', 'Acurácia', 'Precisão', 'Recall', 'F1-Score', 'AUC-ROC', 'CV Mean', 'CV Std']].copy()
df_ranking = df_ranking.sort_values('Acurácia', ascending=False).reset_index(drop=True)
df_ranking.index = df_ranking.index + 1  # Ranking começa em 1

# Formatar para exibição
df_display = df_ranking.copy()
for col in ['Acurácia', 'Precisão', 'Recall', 'F1-Score', 'AUC-ROC', 'CV Mean']:
    df_display[col] = df_display[col].apply(lambda x: f'{x*100:.2f}%')
df_display['CV Std'] = df_display['CV Std'].apply(lambda x: f'±{x*100:.2f}%')

print('='*100)
print('📊 RANKING DOS MODELOS DE MACHINE LEARNING')
print('='*100)
print(df_display.to_string())
print('\n')

# Destacar o melhor
melhor = df_ranking.iloc[0]
print(f'🏆 MELHOR MODELO: {melhor["Modelo"]}')
print(f'   Acurácia: {melhor["Acurácia"]*100:.2f}%')
print(f'   AUC-ROC: {melhor["AUC-ROC"]*100:.2f}%')
print(f'   F1-Score: {melhor["F1-Score"]*100:.2f}%')

In [ ]:
# Visualização comparativa
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Acurácia
df_plot = df_ranking[['Modelo', 'Acurácia']].sort_values('Acurácia')
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.9, len(df_plot)))
bars = axes[0].barh(df_plot['Modelo'], df_plot['Acurácia']*100, color=colors)
axes[0].set_xlabel('Acurácia (%)')
axes[0].set_title('Comparação de Acurácia dos Modelos', fontsize=14, fontweight='bold')
axes[0].set_xlim(0, 100)
for bar, val in zip(bars, df_plot['Acurácia']):
    axes[0].text(val*100 + 1, bar.get_y() + bar.get_height()/2, f'{val*100:.1f}%', va='center', fontsize=9)

# Gráfico 2: Múltiplas métricas
metricas = ['Acurácia', 'Precisão', 'Recall', 'F1-Score']
x = np.arange(len(df_ranking))
width = 0.2

for i, metrica in enumerate(metricas):
    axes[1].bar(x + i*width, df_ranking[metrica]*100, width, label=metrica)

axes[1].set_ylabel('Porcentagem (%)')
axes[1].set_title('Comparação de Métricas por Modelo', fontsize=14, fontweight='bold')
axes[1].set_xticks(x + width*1.5)
axes[1].set_xticklabels(df_ranking['Modelo'], rotation=45, ha='right')
axes[1].legend()
axes[1].set_ylim(0, 100)

plt.tight_layout()
plt.savefig('comparacao_modelos.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Curvas ROC de todos os modelos
plt.figure(figsize=(10, 8))

for resultado in resultados:
    fpr, tpr, _ = roc_curve(y_test, resultado['y_proba'])
    plt.plot(fpr, tpr, lw=2, label=f"{resultado['Modelo']} (AUC={resultado['AUC-ROC']*100:.1f}%)")

plt.plot([0, 1], [0, 1], color='gray', linestyle='--', label='Aleatório')
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curvas ROC - Comparação de Modelos', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=8)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('curvas_roc.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Análise do Modelo Escolhido

Com base na comparação, selecionamos o modelo com melhor desempenho.

In [ ]:
# Selecionar melhor modelo
idx_melhor = df_resultados['Acurácia'].idxmax()
melhor_resultado = df_resultados.iloc[idx_melhor]
modelo_final = melhor_resultado['modelo_obj']
y_pred_final = melhor_resultado['y_pred']
y_proba_final = melhor_resultado['y_proba']

print(f'📊 Relatório de Classificação - {melhor_resultado["Modelo"]}')
print('='*60)
print(classification_report(y_test, y_pred_final, target_names=['Sem Risco', 'Com Risco']))

In [ ]:
# Matriz de Confusão
fig, ax = plt.subplots(figsize=(8, 6))

cm = confusion_matrix(y_test, y_pred_final)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['Sem Risco', 'Com Risco'], yticklabels=['Sem Risco', 'Com Risco'])
ax.set_xlabel('Predito')
ax.set_ylabel('Real')
ax.set_title(f'Matriz de Confusão - {melhor_resultado["Modelo"]}', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('matriz_confusao.png', dpi=150, bbox_inches='tight')
plt.show()

# Interpretação
print('\n📊 Interpretação da Matriz de Confusão:')
print(f'   Verdadeiros Negativos (Sem Risco correto): {cm[0,0]}')
print(f'   Falsos Positivos (Sem Risco predito como Com Risco): {cm[0,1]}')
print(f'   Falsos Negativos (Com Risco predito como Sem Risco): {cm[1,0]}')
print(f'   Verdadeiros Positivos (Com Risco correto): {cm[1,1]}')

In [ ]:
# Feature Importance (usando Random Forest para interpretabilidade)
rf_model = modelos['Random Forest']

# Mapear nomes das features
feature_names = features_final.copy()
for i, name in enumerate(feature_names):
    if '_ENC' in name:
        feature_names[i] = name.replace('_ENC', '')

df_imp = pd.DataFrame({
    'Feature': feature_names,
    'Importância': rf_model.feature_importances_
}).sort_values('Importância', ascending=False)

print('📈 Importância das Features (Random Forest):')
print('='*50)
for _, row in df_imp.iterrows():
    barra = '█' * int(row['Importância'] * 50)
    print(f'{row["Feature"]:20}: {row["Importância"]*100:5.1f}% {barra}')

# Visualizar
fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.Blues(np.linspace(0.4, 0.9, len(df_imp)))
bars = ax.barh(df_imp['Feature'], df_imp['Importância']*100, color=colors)
ax.set_xlabel('Importância (%)')
ax.set_title('Importância das Features para Predição de Risco', fontsize=14, fontweight='bold')
for bar, val in zip(bars, df_imp['Importância']):
    ax.text(val*100 + 0.5, bar.get_y() + bar.get_height()/2, f'{val*100:.1f}%', va='center', fontsize=10)
plt.tight_layout()
plt.savefig('importancia_features.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Salvar Modelo para Deploy (Streamlit)

In [ ]:
# Salvar modelo, scaler e metadados
output_dir = 'streamlit/'
os.makedirs(output_dir, exist_ok=True)

# Salvar modelo
with open(f'{output_dir}modelo_risco_defasagem.pkl', 'wb') as f:
    pickle.dump(modelo_final, f)
print(f'✅ Modelo salvo')

# Salvar scaler
with open(f'{output_dir}scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print(f'✅ Scaler salvo')

# Salvar LabelEncoders
with open(f'{output_dir}label_encoders.pkl', 'wb') as f:
    pickle.dump(le_dict, f)
print(f'✅ LabelEncoders salvos')

# Salvar features
with open(f'{output_dir}features.txt', 'w') as f:
    f.write(','.join(features_final))
print(f'✅ Features salvas')

# Salvar info do modelo
modelo_info = {
    'features': features_final,
    'features_numericas': [f for f in features_final if '_ENC' not in f],
    'features_categoricas': [f.replace('_ENC', '') for f in features_final if '_ENC' in f],
    'classes': {0: 'Sem Risco', 1: 'Com Risco'},
    'accuracy': float(melhor_resultado['Acurácia']),
    'auc_roc': float(melhor_resultado['AUC-ROC']),
    'modelo_nome': melhor_resultado['Modelo'],
    'feature_importance': dict(zip(feature_names, rf_model.feature_importances_.tolist()))
}

with open(f'{output_dir}modelo_info.pkl', 'wb') as f:
    pickle.dump(modelo_info, f)
print(f'✅ Info do modelo salva')

print(f'\n📁 Arquivos salvos em {output_dir}')

## 10. Conclusões

### Comparação de Modelos

Foram testados **10 algoritmos de Machine Learning** diferentes:

| Categoria | Modelos Testados |
|-----------|------------------|
| Lineares | Logistic Regression |
| Baseados em Distância | KNN, SVM |
| Baseados em Árvore | Decision Tree, Random Forest, Gradient Boosting, XGBoost, AdaBoost |
| Redes Neurais | MLP |
| Ensemble | Voting Classifier |

### Por que escolhemos o modelo final?

O modelo foi escolhido com base em:
1. **Maior Acurácia** no conjunto de teste
2. **Melhor AUC-ROC** (capacidade de distinguir entre classes)
3. **Estabilidade** na validação cruzada (baixo desvio padrão)

### Principais Insights

1. **IDADE é o fator mais importante (~38%)** - Funciona como multiplicador de risco
2. **IEG (Engajamento)** é o indicador PEDE mais relevante (~10%)
3. **Notas de Matemática** têm maior poder preditivo que outras matérias

### Limitações

- Os indicadores de desempenho atual capturam apenas parte do risco
- A defasagem é um acúmulo histórico que depende de fatores além do desempenho atual
- O modelo deve ser usado como ferramenta de triagem, não como decisão final

### Recomendações

1. **Intervenção precoce** em alunos jovens com indicadores baixos
2. **Atenção especial** a alunos mais velhos com desempenho médio/baixo
3. **Monitoramento contínuo** dos indicadores IEG e IPV como sinais de alerta